In [1]:
# !pip install transformers deepspeed-mii --upgrade

In [1]:
# create the model
import transformers
from transformers import pipeline

import deepspeed

pipe = pipeline("text-generation", model="bigscience/bloom-560m", device=0)
# Initialize the DeepSpeed-Inference engine


Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [4]:
import torch
pipe.model = deepspeed.init_inference(
    pipe.model,
    mp_size=1,
    dtype=torch.half,
    replace_method='auto',
    replace_with_kernel_inject=True
)

[2022-11-16 19:34:36,055] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.5, git-hash=unknown, git-branch=unknown
[2022-11-16 19:34:36,056] [INFO] [logging.py:68:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
Using /home/ec2-user/.cache/torch_extensions/py38_cu111 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ec2-user/.cache/torch_extensions/py38_cu111/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module transformer_inference...
Time to load transformer_inference op: 0.3223097324371338 seconds
[2022-11-16 19:34:37,038] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layer

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

In [8]:
start_text = "Testing BLOOM-3B with DeepSpeed MII"
tokens_start_text = len(tokenizer(start_text, return_tensors="pt").input_ids[0])
tokens_start_text

11

In [9]:
import time

new_tokens = 500
gen_length = new_tokens + tokens_start_text
t0 = time.time()
result = pipe(start_text, min_length=gen_length, max_length=gen_length)
t1 = time.time()

In [10]:
result

[{'generated_text': 'Testing BLOOM-3B with DeepSpeed MII and the results are shown in Fig. 3. The results show that the BLOOM-3B with DeepSpeed MII can achieve a higher accuracy than the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B wit

In [11]:
gen_text = result[0]['generated_text']
tokens_gen_text = len(tokenizer(gen_text, return_tensors="pt").input_ids[0])

In [13]:
total_new_tokens_generated = tokens_gen_text - tokens_start_text
throughput = (total_new_tokens_generated) / (t1 - t0)
print(f"Tokens generated: {total_new_tokens_generated}; Time: {t1 - t0:.1f} seconds; Tokens per second: {throughput:.1f}; Latency: {1000 / throughput:.1f} ms")

Tokens generated: 500; Time: 7.0 seconds; Tokens per second: 70.9; Latency: 14.1 ms


In [14]:
mii.terminate("bloom-560m-mii")

[2022-11-16 19:23:07,242] [INFO] [terminate.py:8:terminate] Terminating server for bloom-560m-mii
[2022-11-16 19:23:08,055] [INFO] [launch.py:318:sigkill_handler] Killing subprocess 4589
[2022-11-16 19:23:08,056] [ERROR] [launch.py:324:sigkill_handler] ['/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python', '-m', 'mii.launch.multi_gpu_server', '--task-name', 'text-generation', '--model', 'bigscience/bloom-560m', '--model-path', '/tmp/mii_models', '--port', '50050', '--ds-optimize', '--provider', 'hugging-face', '--config', 'eyJ0ZW5zb3JfcGFyYWxsZWwiOiAxLCAicG9ydF9udW1iZXIiOiA1MDA1MCwgImR0eXBlIjogImZwMzIiLCAiZW5hYmxlX2N1ZGFfZ3JhcGgiOiBmYWxzZSwgImNoZWNrcG9pbnRfZGljdCI6IG51bGwsICJkZXBsb3lfcmFuayI6IFswXSwgInRvcmNoX2Rpc3RfcG9ydCI6IDI5NTAwLCAiaGZfYXV0aF90b2tlbiI6IG51bGwsICJyZXBsYWNlX3dpdGhfa2VybmVsX2luamVjdCI6IHRydWUsICJwcm9maWxlX21vZGVsX3RpbWUiOiBmYWxzZSwgInNraXBfbW9kZWxfY2hlY2siOiBmYWxzZX0='] exits with return code = -15
